In [1]:
import csv
import os
import urllib3
from lxml import etree
import re
import codecs
import sys
import xml.etree.ElementTree as ET
import pandas as pd
import shutil
import glob

### Initialisation

In [2]:
data = pd.read_csv('arkissue.tsv',sep='\t') #J'importe la liste des identifiants ARK que j'avais déjà récupéré précédemment.
list_ark_all = data["ID"].to_list()

In [3]:
# Je supprime les cas spéciaux (numéros incomplets sur Gallica)
special_ark = ['bpt6k1330971w', 'bpt6k1330975j', 'bpt6k13309729', 'bpt6k13309907', 'bpt6k1330973q', 'bpt6k13308489', 'bpt6k1330873c']
list_ark_all = list(set(list_ark_all)-set(special_ark))

In [39]:
list_file_done = glob.glob('./OUTPUT/*.txt') # Certains téléchargements ne s'effectuant pas, j'ai fait en plusieurs étapes
list_ark_done = [] # en déplacant les fichiers TXT qui étaient complet dans un dossier CORPUS

In [41]:
# Ainsi, je crée une liste des ID ARK traités
myre = '(bpt\w{7,10})'
for i in list_file_done:
    if re.search(myre, i):
        list_ark_done.append(re.search(myre, i).group(1))

In [42]:
list_ark_all = list(set(list_ark_all)-set(list_ark_done))
# Cette liste soustrait les ARK traités de la totalité des ARK

In [43]:
list_ark_20 = list_ark_all[:10] # 238 est un multiple de 14
list_ark_20

['bpt6k1330879v',
 'bpt6k13308133',
 'bpt6k13309974',
 'bpt6k13309032',
 'bpt6k13307783',
 'bpt6k1330881x',
 'bpt6k13308845',
 'bpt6k1330962x',
 'bpt6k1330837h',
 'bpt6k1330936f']

In [44]:
folio = ['1','2', '3', '4'] # Nomrbe de pages de presses

**Ark effectués :**
- 1er jet *(26/09/2022)* : 0 -> 20, test et dev du script sans tri de la liste des ARK
- 2e jet *(26/09/2022)* : 20 -> 25, test traitement des images par Python sans tri de la liste des ARK
- 3e jet *(26/09/2022)* : 20, comparaison rapide des qualités d'OCR entre images process et non process
- 4e jet *(27/09/2022)* : étude approfondie pour connaître la précision des OCR selon le degré de traitement des images
- 5e jet *(27/09/2022)* : premiers jets sérieux

In [45]:
len(list_ark_all)

221

### Téléchargement des fichiers ALTO 

In [10]:
def download_alto_file(i, n):  # Création d'une fonction pour récupérer les fichiers ALTO fournis par Gallica
    http = urllib3.PoolManager() 
    request = http.request('GET', "https://gallica.bnf.fr/RequestDigitalElement?O=" + i + "&E=ALTO&Deb=" + n) 
    xml = request.data
    filename = i + '_' + "p" + n +".xml"
    with open(filename, "wb") as f:
        f.write(xml)

In [11]:
for i in list_ark_20: # Boucle utilisant la fonction 
    for y in folio:
        download_alto_file(i, y)

In [12]:
parent = os.getcwd()

### Gestion des fichiers : passage des fichiers ALTO en UZN puis import des JPG correspondantent

In [13]:
xml_files = [] # Création d'une liste de tous les fichiers XML
for i in glob.glob("*.xml"):
    xml_files.append(i)

In [14]:
for f in xml_files: #Pour chaque XML, le récupère les métadonnées relatives aux coordonnées de chaque TextBlock
    if sys.stdout.encoding != 'UTF-8': # que j'implémente dans un fichier UZn nouvellement créé
        sys.stdout = codecs.getwriter('utf-8')(sys.stdout.buffer, 'strict')

    namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
                 'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
                 'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#'}
    tree = ET.parse(f)
    xmlns = tree.getroot().tag.split('}')[0].strip('{')
    if xmlns in namespace.values():
        with open(f + ".uzn", "a+") as f:
            for line in tree.iterfind('.//{%s}TextBlock' % xmlns):
                #sys.stdout.write('\n')
                text = line.attrib.get('HPOS') + ' ' + line.attrib.get('VPOS') + ' ' + line.attrib.get('WIDTH') + ' ' + line.attrib.get('HEIGHT') +  ' ' + line.attrib.get('ID') + '\n'
                f.write(text)
    else:
        print('ERROR: Not a valid ALTO file (namespace declaration missing)')

In [15]:
uzn_files = [] # Liste de tous les fichiers UZN avec les noms uniformisés
for u in glob.glob("*.uzn"):
    uzn_files.append(u)
for u in uzn_files:
    y = re.sub(".xml", '', u)
    os.rename(u,y)
uzn_files.sort()

In [16]:
for xml in xml_files: # Suppression des fichiers XML pour gagner de l'espace
    os.remove(xml)

In [17]:
def move_pages_to_parent(page):# Enfin, on se déplace dans les dossiers PAGES afin de récupérer
    os.chdir(parent + '/pages_' + page) #les fichiers JPG dans le dossier correspondant en se basant sur le code ARK
    filenames = os.listdir() 
    for name in list_ark_20:
        for file in filenames:
            if file.endswith(name + '_p' + page + '.jpg'):
                os.rename(file, parent + '/' + file)
    os.chdir(parent)

In [18]:
for p in folio:
    move_pages_to_parent(p)

In [19]:
jpg_files = [] # On liste tous les fichiers JPG
for i in glob.glob('*.jpg'):
    jpg_files.append(i)
jpg_files.sort()

In [20]:
for i in list_ark_20: # On supprime les fichiers UZN des pages 4 si les JPG_p4 n'existent pas
    page = i + "_p4.jpg" # La liste des page 4 contenant du texte pertinent a été préalablement faite
    uznf = i + "_p4.uzn"
    if page not in jpg_files:
        os.remove(uznf)

In [21]:
uzn_files = []  # Je le refais pour être sûr que les noms de la liste UZN sont bons
for u in glob.glob("*.uzn"):
    uzn_files.append(u)
uzn_files.sort()

In [22]:
jpg_files # test

['bpt6k13307805_p1.jpg',
 'bpt6k13307805_p2.jpg',
 'bpt6k13307805_p3.jpg',
 'bpt6k13307805_p4.jpg',
 'bpt6k1330800g_p1.jpg',
 'bpt6k1330800g_p2.jpg',
 'bpt6k1330800g_p3.jpg',
 'bpt6k1330800g_p4.jpg',
 'bpt6k1330808s_p1.jpg',
 'bpt6k1330808s_p2.jpg',
 'bpt6k1330808s_p3.jpg',
 'bpt6k1330808s_p4.jpg',
 'bpt6k13308437_p1.jpg',
 'bpt6k13308437_p2.jpg',
 'bpt6k13308437_p3.jpg',
 'bpt6k1330865t_p1.jpg',
 'bpt6k1330865t_p2.jpg',
 'bpt6k1330865t_p3.jpg',
 'bpt6k1330874s_p1.jpg',
 'bpt6k1330874s_p2.jpg',
 'bpt6k1330874s_p3.jpg',
 'bpt6k1330874s_p4.jpg',
 'bpt6k13309581_p1.jpg',
 'bpt6k13309581_p2.jpg',
 'bpt6k13309581_p3.jpg',
 'bpt6k1330991n_p1.jpg',
 'bpt6k1330991n_p2.jpg',
 'bpt6k1330991n_p3.jpg',
 'bpt6k1330993g_p1.jpg',
 'bpt6k1330993g_p2.jpg',
 'bpt6k1330993g_p3.jpg',
 'bpt6k13310127_p1.jpg',
 'bpt6k13310127_p2.jpg',
 'bpt6k13310127_p3.jpg']

In [23]:
uzn_files # test

['bpt6k13307805_p1.uzn',
 'bpt6k13307805_p2.uzn',
 'bpt6k13307805_p3.uzn',
 'bpt6k13307805_p4.uzn',
 'bpt6k1330800g_p1.uzn',
 'bpt6k1330800g_p2.uzn',
 'bpt6k1330800g_p3.uzn',
 'bpt6k1330800g_p4.uzn',
 'bpt6k1330808s_p1.uzn',
 'bpt6k1330808s_p2.uzn',
 'bpt6k1330808s_p3.uzn',
 'bpt6k1330808s_p4.uzn',
 'bpt6k13308437_p1.uzn',
 'bpt6k13308437_p2.uzn',
 'bpt6k13308437_p3.uzn',
 'bpt6k1330865t_p1.uzn',
 'bpt6k1330865t_p2.uzn',
 'bpt6k1330865t_p3.uzn',
 'bpt6k1330874s_p1.uzn',
 'bpt6k1330874s_p2.uzn',
 'bpt6k1330874s_p3.uzn',
 'bpt6k1330874s_p4.uzn',
 'bpt6k13309581_p1.uzn',
 'bpt6k13309581_p2.uzn',
 'bpt6k13309581_p3.uzn',
 'bpt6k1330991n_p1.uzn',
 'bpt6k1330991n_p2.uzn',
 'bpt6k1330991n_p3.uzn',
 'bpt6k1330993g_p1.uzn',
 'bpt6k1330993g_p2.uzn',
 'bpt6k1330993g_p3.uzn',
 'bpt6k13310127_p1.uzn',
 'bpt6k13310127_p2.uzn',
 'bpt6k13310127_p3.uzn']

### Traitement des images

In [24]:
from wand.image import Image
import cv2
import numpy as np

In [25]:
#Thresholding

for jpg in jpg_files:
    with Image(filename = jpg) as img:
        with img.clone() as otsu:
            otsu.auto_threshold(method='otsu')
            otsu.save(filename= jpg)

In [26]:
#Despeckling

for jpg in jpg_files:
    with Image(filename=jpg) as img:
        img.despeckle()
        img.save(filename=jpg)

### Maintenant, place à la transcription

In [27]:
for i in range(len(jpg_files)): # test de correspondance
    print(jpg_files[i] + ' ' + uzn_files[i])

bpt6k13307805_p1.jpg bpt6k13307805_p1.uzn
bpt6k13307805_p2.jpg bpt6k13307805_p2.uzn
bpt6k13307805_p3.jpg bpt6k13307805_p3.uzn
bpt6k13307805_p4.jpg bpt6k13307805_p4.uzn
bpt6k1330800g_p1.jpg bpt6k1330800g_p1.uzn
bpt6k1330800g_p2.jpg bpt6k1330800g_p2.uzn
bpt6k1330800g_p3.jpg bpt6k1330800g_p3.uzn
bpt6k1330800g_p4.jpg bpt6k1330800g_p4.uzn
bpt6k1330808s_p1.jpg bpt6k1330808s_p1.uzn
bpt6k1330808s_p2.jpg bpt6k1330808s_p2.uzn
bpt6k1330808s_p3.jpg bpt6k1330808s_p3.uzn
bpt6k1330808s_p4.jpg bpt6k1330808s_p4.uzn
bpt6k13308437_p1.jpg bpt6k13308437_p1.uzn
bpt6k13308437_p2.jpg bpt6k13308437_p2.uzn
bpt6k13308437_p3.jpg bpt6k13308437_p3.uzn
bpt6k1330865t_p1.jpg bpt6k1330865t_p1.uzn
bpt6k1330865t_p2.jpg bpt6k1330865t_p2.uzn
bpt6k1330865t_p3.jpg bpt6k1330865t_p3.uzn
bpt6k1330874s_p1.jpg bpt6k1330874s_p1.uzn
bpt6k1330874s_p2.jpg bpt6k1330874s_p2.uzn
bpt6k1330874s_p3.jpg bpt6k1330874s_p3.uzn
bpt6k1330874s_p4.jpg bpt6k1330874s_p4.uzn
bpt6k13309581_p1.jpg bpt6k13309581_p1.uzn
bpt6k13309581_p2.jpg bpt6k13309581

In [28]:
for i in range(len(jpg_files)): # On lance la commande système pour effectuer l'OCR
    os.system('tesseract ' + jpg_files[i] + ' ' + uzn_files[i] + ' --psm 4 -l fra+cos+ita')

UZN file bpt6k13307805_p1.uzn loaded.
UZN file bpt6k13307805_p2.uzn loaded.
UZN file bpt6k13307805_p3.uzn loaded.
UZN file bpt6k13307805_p4.uzn loaded.
UZN file bpt6k1330800g_p1.uzn loaded.
UZN file bpt6k1330800g_p2.uzn loaded.
UZN file bpt6k1330800g_p3.uzn loaded.
UZN file bpt6k1330800g_p4.uzn loaded.
UZN file bpt6k1330808s_p1.uzn loaded.
UZN file bpt6k1330808s_p2.uzn loaded.
UZN file bpt6k1330808s_p3.uzn loaded.
UZN file bpt6k1330808s_p4.uzn loaded.
UZN file bpt6k13308437_p1.uzn loaded.
UZN file bpt6k13308437_p2.uzn loaded.
UZN file bpt6k13308437_p3.uzn loaded.
UZN file bpt6k1330865t_p1.uzn loaded.
UZN file bpt6k1330865t_p2.uzn loaded.
UZN file bpt6k1330865t_p3.uzn loaded.
UZN file bpt6k1330874s_p1.uzn loaded.
UZN file bpt6k1330874s_p2.uzn loaded.
UZN file bpt6k1330874s_p3.uzn loaded.
UZN file bpt6k1330874s_p4.uzn loaded.
UZN file bpt6k13309581_p1.uzn loaded.
UZN file bpt6k13309581_p2.uzn loaded.
UZN file bpt6k13309581_p3.uzn loaded.
UZN file bpt6k1330991n_p1.uzn loaded.
UZN file bpt

### Pour terminer, classons tout ça dans les bons dossiers en les rendant plus propres

In [29]:
for jpg in jpg_files: # On supprime les fichiers JPG pour gagner en lisibilité
    os.remove(jpg)

In [30]:
for uzn in uzn_files: # On fait de même avec les fichiers UZN
    os.remove(uzn)

In [31]:
txt_files = [] # Comme pour les fichiers UZN plus tôt, on listes les fichiers TXT et on uniformise les noms
for t in glob.glob('*.txt'):
    txt_files.append(t)
for t in txt_files:
    y = re.sub(".uzn", "", t)
    os.rename(t,y)
txt_files = []
for t in glob.glob('*.txt'):
    txt_files.append(t)

In [32]:
txt_files.sort() # On trie (toujours trier pour être sûr)

In [33]:
for i in list_ark_20: # On crée des dossiers correspondant aux numéros traités présentement
    os.system("mkdir " + i)

In [34]:
for file in txt_files: # On regroupe dans ces dossiers tous les fichiers TXT du même numéro
    for ark in list_ark_20:
        if file.startswith(ark):
            os.rename(file, parent + "/" + ark + "/" + file)

In [35]:
for folder in list_ark_20: # On lance une commande BASH pour concaténer les fichiers TXT dans un seul
    os.chdir(parent + '/' + folder)
    os.system("cat *.txt > " + folder + ".txt")

In [36]:
for folder in list_ark_20: # On supprime tous les TXT des pages unique spour ne laisser que le TXT global
    os.chdir(parent + '/' + folder)
    filenames = os.listdir()
    for file in filenames:
        if re.search("_p", file):
            os.remove(file)

In [37]:
for folder in list_ark_20: # On déplace tous les fichiers TXT dans les dans un dossier commun OUTPUT
    os.chdir(parent + '/' + folder)
    os.rename(folder + ".txt", parent + "/OUTPUT/" + folder + ".txt")

In [38]:
for folder in list_ark_20: # On supprime les dossiers vides
    os.chdir(parent)
    os.rmdir(folder)

### Commentaires

Il y a beaucoup de cellules différentes mais c'est pour ne pas se précipiter non plus : on ne sait jamais, s'il y a un problème, on peut toujours le régler si on sait quand il survient.

La dernière phase de concaténation des fichiers `.txt` pourrait être simplifier mais je préfère manipuler ces fichiers dans des dossiers séparés pour éviter des incidents et la concaténation de fichiers qui ne sont pas issus des mêmes numéros. 

### Petits scripts bonus

In [ ]:
for i in jpg_files:
    y = re.sub("recup_transcription", '', i)
    os.rename(i,y)

In [ ]:
for i in list_ark_20:
    os.system("mkdir " + i)